<a href="https://colab.research.google.com/github/annisafitribas/ft_credit_home/blob/main/ft_credit_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PERSIAPAN**

In [4]:
import os, sys, time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

WORKDIR = '/content/home_credit_task'
os.makedirs(WORKDIR, exist_ok=True)
print("WORKDIR:", WORKDIR)

WORKDIR: /content/home_credit_task


Bertujuan untuk menyiapkan folder kerja dan library yang akan digunakan dalam proses pengolahan data

In [5]:
# install dependencies if missing (Colab-friendly)
try:
    import gdown
except Exception:
    !pip install -q gdown
    import gdown

try:
    import lightgbm as lgb
except Exception:
    !pip install -q lightgbm
    import lightgbm as lgb

try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
except Exception:
    !pip install -q python-pptx
    from pptx import Presentation
    from pptx.util import Inches, Pt

try:
    import matplotlib
    import matplotlib.pyplot as plt
except Exception:
    !pip install -q matplotlib
    import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.5 MB/s eta 0:00:00


In [6]:
# sklearn / joblib
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.impute import SimpleImputer
import joblib
import pandas as pd
import numpy as np
from pathlib import Path

# 1. Download dataset file

In [7]:
files = {
    'application_train.csv': '1q059QolR6CNxB0PWESAjkEWIprNutajA',
    'application_test.csv' : '1QD7ehk_hzXze0vHQuYa5qyqfDcfI8Sex',
    'bureau.csv'           : '1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF',
    'bureau_balance.csv'   : '1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x',
    'credit_card_balance.csv': '1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu',
    'installments_payments.csv': '126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8',
    'POS_CASH_balance.csv' : '1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA',
    'previous_application.csv': '1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU',
    'HomeCredit_columns_description.csv': '1v2iGGOJjlUGSTsQz-bsjtjtyM5IQp7uW',
    'sample_submission.csv': '1JongVA9fWMYml5XKVnbhm8TUlR5Efs0n'
}

for fname, fid in files.items():
    dest = os.path.join(WORKDIR, fname)
    if not os.path.exists(dest):
        print("Downloading", fname)
        url = f"https://drive.google.com/uc?export=download&id={fid}"
        gdown.download(url, dest, quiet=False)
    else:
        print("Exists:", fname)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1q059QolR6CNxB0PWESAjkEWIprNutajA
From (redirected): https://drive.google.com/uc?export=download&id=1q059QolR6CNxB0PWESAjkEWIprNutajA&confirm=t&uuid=8e53a875-f2de-4aa0-ac59-7f87ff9780b0
To: /content/home_credit_task/application_train.csv
100%|██████████| 166M/166M [00:02<00:00, 65.3MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1QD7ehk_hzXze0vHQuYa5qyqfDcfI8Sex
To: /content/home_credit_task/application_test.csv
100%|██████████| 26.6M/26.6M [00:00<00:00, 59.5MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF
From (redirected): https://drive.google.com/uc?export=download&id=1hndizX1t5ab0DTnKMTedqVJ1ZxLVclhF&confirm=t&uuid=b7e384b4-f151-48bb-82b7-a6a55afe6075
To: /content/home_credit_task/bureau.csv
100%|██████████| 170M/170M [00:01<00:00, 91.9MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x
From (redirected): https://drive.google.com/uc?export=download&id=1OXEQb_L6S_mZALJi4--C6RyFI6yOsq4x&confirm=t&uuid=db79f246-e1da-4428-a87b-bb322a7a2289
To: /content/home_credit_task/bureau_balance.csv
100%|██████████| 376M/376M [00:03<00:00, 112MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu
From (redirected): https://drive.google.com/uc?export=download&id=1t6Hhsmj0vSCCKUlNXht_xDQ6Z6l4M0Vu&confirm=t&uuid=f21ffd15-c000-4085-b52a-9bf9c228165a
To: /content/home_credit_task/credit_card_balance.csv
100%|██████████| 425M/425M [00:04<00:00, 91.9MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8
From (redirected): https://drive.google.com/uc?export=download&id=126xrKCW5EQrxkQoDwmN-yb00ILBKnhR8&confirm=t&uuid=1ca0f61c-e6c9-456d-ac7b-a785dc4e912e
To: /content/home_credit_task/installments_payments.csv
100%|██████████| 723M/723M [00:08<00:00, 83.0MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA
From (redirected): https://drive.google.com/uc?export=download&id=1dODAmBQLaylpM2JcCHfc4KNbbtKY7xhA&confirm=t&uuid=7b7d2aef-1db2-4ae1-ba33-8f61f73ad537
To: /content/home_credit_task/POS_CASH_balance.csv
100%|██████████| 393M/393M [00:04<00:00, 91.3MB/s]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU
From (redirected): https://drive.google.com/uc?export=download&id=1D4O7xf-lF_3oBeu6XMwzhpXtSvhcgoBU&confirm=t&uuid=47e9c7ae-672c-4559-a791-9415665bd148
To: /content/home_credit_task/previous_application.csv
100%|██████████| 405M/405M [00:03<00:00, 107MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1v2iGGOJjlUGSTsQz-bsjtjtyM5IQp7uW
To: /content/home_credit_task/HomeCredit_columns_description.csv
100%|██████████| 37.4k/37.4k [00:00<00:00, 38.8MB/s]


Downloading...
From: https://drive.google.com/uc?export=download&id=1JongVA9fWMYml5XKVnbhm8TUlR5Efs0n
To: /content/home_credit_task/sample_submission.csv
100%|██████████| 536k/536k [00:00<00:00, 62.7MB/s]


# Load csv

In [8]:
print("\nLoading CSVs ...")
train = pd.read_csv(os.path.join(WORKDIR, 'application_train.csv'), low_memory=False)
test  = pd.read_csv(os.path.join(WORKDIR, 'application_test.csv'), low_memory=False)
bureau = pd.read_csv(os.path.join(WORKDIR, 'bureau.csv'), low_memory=False)
bureau_balance = pd.read_csv(os.path.join(WORKDIR, 'bureau_balance.csv'), low_memory=False)
credit_card_balance = pd.read_csv(os.path.join(WORKDIR, 'credit_card_balance.csv'), low_memory=False)
installments = pd.read_csv(os.path.join(WORKDIR, 'installments_payments.csv'), low_memory=False)
pos_cash = pd.read_csv(os.path.join(WORKDIR, 'POS_CASH_balance.csv'), low_memory=False)
prev_app = pd.read_csv(os.path.join(WORKDIR, 'previous_application.csv'), low_memory=False)
sample_sub = pd.read_csv(os.path.join(WORKDIR, 'sample_submission.csv'), low_memory=False)

print("Shapes:")
print("train", train.shape, "test", test.shape)
print("bureau", bureau.shape, "bureau_balance", bureau_balance.shape)
print("credit_card_balance", credit_card_balance.shape, "installments", installments.shape)
print("pos_cash", pos_cash.shape, "previous_application", prev_app.shape)


Loading CSVs ...
Shapes:
train (307511, 122) test (48744, 121)
bureau (1716428, 17) bureau_balance (27299925, 3)
credit_card_balance (3840312, 23) installments (13605401, 8)
pos_cash (10001358, 8) previous_application (1670214, 37)


# **3. Quick EDA & visuals**

In [9]:
print("\n--- TARGET distribution ---")
print(train['TARGET'].value_counts(normalize=True))

# helper for saving plots
def savefig(fig, filename):
    path = os.path.join(WORKDIR, filename)
    fig.savefig(path, bbox_inches='tight')
    print('Saved plot:', path)


--- TARGET distribution ---
TARGET
0    0.919271
1    0.080729
Name: proportion, dtype: float64


## 3.1 target distribution plot

In [10]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
counts = train['TARGET'].value_counts().sort_index()
ax.bar(counts.index.astype(str), counts.values)
ax.set_title('Target distribution (counts)')
ax.set_xlabel('TARGET')
ax.set_ylabel('Count')
savefig(fig, 'target_distribution.png')
plt.close(fig)

Saved plot: /content/home_credit_task/target_distribution.png


## 3.2 top missing features (bar)

In [11]:
missing = train.isna().mean().sort_values(ascending=False).head(30)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.barh(missing.index[::-1], missing.values[::-1])
ax.set_title('Top missing percentage (train)')
ax.set_xlabel('Fraction missing')
savefig(fig, 'missing_pct_top.png')
plt.close(fig)

Saved plot: /content/home_credit_task/missing_pct_top.png


## 3.3 correlation heatmap of numeric features (sampled for speed)

In [12]:
num = train.select_dtypes(include=[np.number]).drop(['SK_ID_CURR','TARGET'], axis=1, errors='ignore')
# sample columns to avoid huge matrix
num_small = num.sample(n=min(30, num.shape[1]), axis=1)
corr = num_small.corr()
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
cax = ax.imshow(corr.values, interpolation='nearest')
ax.set_xticks(np.arange(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90, fontsize=8)
ax.set_yticks(np.arange(len(corr.columns)))
ax.set_yticklabels(corr.columns, fontsize=8)
ax.set_title('Correlation matrix (subset)')
fig.colorbar(cax, ax=ax)
savefig(fig, 'corr_matrix_subset.png')
plt.close(fig)

Saved plot: /content/home_credit_task/corr_matrix_subset.png


# **4. Feature engineering (same as baseline but kept clear)**

## 4.1 Aggregations dari bureau (per SK_ID_CURR)

In [13]:
b_agg = bureau.groupby('SK_ID_CURR').agg(
    bureau_loans_count = ('SK_ID_BUREAU', 'count'),
    bureau_credit_sum_mean = ('AMT_CREDIT_SUM', 'mean'),
    bureau_credit_sum_max = ('AMT_CREDIT_SUM', 'max'),
    bureau_active_cnt = ('CREDIT_ACTIVE', lambda x: (x=='Active').sum())
).reset_index()

## 4.2 bureau_balance -> bad rate per bureau id then agg

In [14]:
bb_bad = bureau_balance[bureau_balance['STATUS'].isin(['2','3','4','5'])].groupby('SK_ID_BUREAU').size().rename('bad_months')
bb_tot = bureau_balance.groupby('SK_ID_BUREAU').size().rename('total_months')
bb = pd.concat([bb_bad, bb_tot], axis=1).fillna(0)
bb['bad_rate'] = bb['bad_months'] / bb['total_months']
bureau2 = bureau.merge(bb.reset_index(), on='SK_ID_BUREAU', how='left')
b2_agg = bureau2.groupby('SK_ID_CURR').agg(
    bureau_prev_bad_rate_mean = ('bad_rate','mean'),
    bureau_prev_months_mean = ('total_months','mean')
).reset_index()

b_agg = b_agg.merge(b2_agg, on='SK_ID_CURR', how='left')

## 4.3 previous_application aggregates

In [15]:
prev_agg = prev_app.groupby('SK_ID_CURR').agg(
    prev_count = ('SK_ID_PREV','count'),
    prev_amt_app_mean = ('AMT_APPLICATION','mean'),
    prev_amt_credit_mean = ('AMT_CREDIT','mean'),
    prev_approved = ('NAME_CONTRACT_STATUS', lambda x: (x=='Approved').sum())
).reset_index()

## # 4.4 installments

In [ ]:
inst_agg = installments.groupby('SK_ID_CURR').agg(
    inst_count = ('NUM_INSTALMENT_VERSION','count'),
    inst_amt_sum = ('AMT_PAYMENT','sum'),
    inst_delay_mean = ('DAYS_ENTRY_PAYMENT', lambda x: np.nanmean(x - installments.loc[x.index,'DAYS_INSTALMENT']))
).reset_index()

## 4.5 credit_card & pos

In [ ]:
cc_agg = credit_card_balance.groupby('SK_ID_CURR').agg(
    cc_count = ('SK_ID_PREV','count'),
    cc_bal_mean = ('AMT_BALANCE','mean'),
    cc_limit_mean = ('AMT_CREDIT_LIMIT_ACTUAL','mean')
).reset_index()

pos_agg = pos_cash.groupby('SK_ID_CURR').agg(
    pos_count = ('SK_ID_PREV','count'),
    pos_dpd_mean = ('SK_DPD','mean')
).reset_index()

## 4.6 application-level features

In [ ]:
def make_app_features(df):
    df = df.copy()
    df['DAYS_BIRTH_YEARS'] = (-df['DAYS_BIRTH']) / 365.25
    df['DAYS_EMPLOYED_YEARS'] = df['DAYS_EMPLOYED'].replace(365243, np.nan) / -365.25
    df['INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / (df['AMT_CREDIT'] + 1)
    df['CREDIT_GOODS_RATIO'] = df['AMT_CREDIT'] / (df['AMT_GOODS_PRICE'] + 1)
    return df[['SK_ID_CURR','DAYS_BIRTH_YEARS','DAYS_EMPLOYED_YEARS','AMT_INCOME_TOTAL','AMT_CREDIT','INCOME_CREDIT_RATIO','CREDIT_GOODS_RATIO']]

app_train_feats = make_app_features(train)
app_test_feats = make_app_features(test)